#### Imports

In [254]:
import os
import pandas as pd
import date_range_data_extractor

## Setup environment

In [255]:
Y = '2022'
quoter = 4

if quoter == 1:
    Q = ['Q1','01-01','03-31']
elif quoter == 2:
    Q = ['Q2','04-01','06-30']
elif quoter == 3:
    Q = ['Q3','07-01','09-30']
elif quoter == 4:
    Q = ['Q4','10-01','12-31']

from_date = f'{Y}-{Q[1]}' # ex: '2021-01-01'
to_date = f'{Y}-{Q[2]}' # ex: '2021-03-31'
output_filename = f'{Y}-{Q[0]}_seconds.csv' # ex: '2021-Q1_seconds.csv'

## Read data

In [256]:
# Extract data from csv files.
data_extractor = date_range_data_extractor.DateRangeDataExtractor()
data_extractor.extract_data(r'./files/', from_date, to_date)
data = data_extractor.data

Loading file 2022-10-01.csv...
Loading file 2022-10-02.csv...
Loading file 2022-10-03.csv...
Loading file 2022-10-04.csv...
Loading file 2022-10-05.csv...
Loading file 2022-10-06.csv...
Loading file 2022-10-07.csv...
Loading file 2022-10-08.csv...
Loading file 2022-10-09.csv...
Loading file 2022-10-10.csv...
Loading file 2022-10-11.csv...
Loading file 2022-10-12.csv...
Loading file 2022-10-13.csv...
Loading file 2022-10-14.csv...
Loading file 2022-10-15.csv...
Loading file 2022-10-16.csv...
Loading file 2022-10-17.csv...
Loading file 2022-10-18.csv...
Loading file 2022-10-19.csv...
Loading file 2022-10-20.csv...
Loading file 2022-10-21.csv...
Loading file 2022-10-22.csv...
Loading file 2022-10-23.csv...
Loading file 2022-10-24.csv...
Loading file 2022-10-25.csv...
Loading file 2022-10-26.csv...
Loading file 2022-10-27.csv...
Loading file 2022-10-28.csv...
Loading file 2022-10-29.csv...
Loading file 2022-10-30.csv...
Loading file 2022-10-31.csv...
Loading file 2022-11-01.csv...
Loading 

Concatenate data and show info

In [257]:
df = pd.concat(data, ignore_index=True, join='inner')
print(df.info(), end='\n\n')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79505934 entries, 0 to 79505933
Data columns (total 2 columns):
 #   Column  Dtype  
---  ------  -----  
 0   Time    object 
 1   Value   float64
dtypes: float64(1), object(1)
memory usage: 1.2+ GB
None


## Scale data
Change the Time value to datetime format and filter the data to whole seconds and remove all other values.

In [258]:
df['Time'] = pd.to_datetime(df['Time'])
df = df[df['Time'].dt.microsecond == 0]
df.reset_index(drop=True, inplace=True)
print(df.head(5))

                 Time     Value
0 2022-10-01 00:00:00  49.98771
1 2022-10-01 00:00:01  49.98478
2 2022-10-01 00:00:02  49.98796
3 2022-10-01 00:00:03  49.98621
4 2022-10-01 00:00:04  49.98732


Add timezone to data

In [259]:
df['Time'] = pd.to_datetime(df['Time']).dt.tz_localize('Europe/Helsinki', ambiguous='infer')
print(df.head(5))

                       Time     Value
0 2022-10-01 00:00:00+03:00  49.98771
1 2022-10-01 00:00:01+03:00  49.98478
2 2022-10-01 00:00:02+03:00  49.98796
3 2022-10-01 00:00:03+03:00  49.98621
4 2022-10-01 00:00:04+03:00  49.98732


Add extra hour

In [260]:
if quoter == 4:
    data_extractor_extra = date_range_data_extractor.DateRangeDataExtractor()
    data_extractor_extra.extract_data(r'./files/', f'{str(int(Y) + 1)}-01-01', f'{str(int(Y) + 1)}-01-01')
    extra_data = data_extractor_extra.data
    edf = pd.concat(extra_data, ignore_index=True, join='inner')
    edf['Time'] = pd.to_datetime(edf['Time'])
    edf = edf[edf['Time'].dt.microsecond == 0]
    edf.reset_index(drop=True, inplace=True)
    edf['Time'] = pd.to_datetime(edf['Time']).dt.tz_localize('Europe/Helsinki', ambiguous='infer')
    edf = edf[edf['Time'] <= f'{str(int(Y) + 1)}-01-01 01:00:00']
    df = pd.concat([df, edf])
    df.reset_index(drop=True, inplace=True)
else:
    print('Skipping!')

Loading file 2023-01-01.csv...


In [261]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7954202 entries, 0 to 7954201
Data columns (total 2 columns):
 #   Column  Dtype                          
---  ------  -----                          
 0   Time    datetime64[ns, Europe/Helsinki]
 1   Value   float64                        
dtypes: datetime64[ns, Europe/Helsinki](1), float64(1)
memory usage: 121.4 MB


## Analyze integrity of data

#### Check and drop duplicates if any

In [262]:
duplicated_rows = df[df.duplicated()]
num_duplicated = len(duplicated_rows)
print(f'Duplicates: \n{num_duplicated}\n{duplicated_rows}', end='\n\n\n')

Duplicates: 
11
                             Time     Value
3601    2022-10-01 01:00:00+03:00  49.96100
14403   2022-10-01 04:00:00+03:00  50.01071
2684942 2022-11-01 01:00:00+02:00  49.93320
2688531 2022-11-01 02:00:00+02:00  50.05831
2692132 2022-11-01 03:00:00+02:00  50.03057
2695733 2022-11-01 04:00:00+02:00  50.05378
5276480 2022-12-01 01:00:00+02:00  49.95108
5280081 2022-12-01 02:00:00+02:00  49.99757
5283682 2022-12-01 03:00:00+02:00  50.03777
5287283 2022-12-01 04:00:00+02:00  50.04827
7954201 2023-01-01 01:00:00+02:00  49.95928



Remove duplicates

In [263]:
df = df.drop_duplicates()

##### Find duplicates in date

In [264]:
duplicated_rows_time = df[df.duplicated(subset="Time", keep=False)]
num_duplicated_time = len(duplicated_rows_time)
print(f'Duplicates in Time column: \n{num_duplicated_time}\n{duplicated_rows_time}', end='\n')

Duplicates in Time column: 
4
                           Time     Value
7201  2022-10-01 02:00:00+03:00  50.01313
7202  2022-10-01 02:00:00+03:00  50.01300
10801 2022-10-01 03:00:00+03:00  49.92700
10802 2022-10-01 03:00:00+03:00  49.92590


Remove duplicates

In [265]:
df = df.drop_duplicates(subset='Time')

#### Any NaN, Null, 0 or "" found in Value.

In [266]:
dfNaN = df[df['Value'].isna()]
nrNaN = len(dfNaN)
dfNull = df[df['Value'].isnull()]
nrNull = len(dfNull)
dfZeroValues = df[df['Value'] == 0]
zeroValues = len(dfZeroValues)
dfWhiteSpaces = df[df['Value'] == ""]
whiteSpaces = len(dfWhiteSpaces)
print(f'Is not a number in value column:: \n{nrNaN}', end='\n')
print(f'Is a NULL in value column:: \n{nrNull}', end='\n')
print(f'Zero values in value column: \n{zeroValues}', end='\n')
print(f'White spaces in value column: \n{whiteSpaces}', end='\n')

Is not a number in value column:: 
0
Is a NULL in value column:: 
0
Zero values in value column: 
0
White spaces in value column: 
0


#### Finding any missing date in the series of dates

In [267]:
df.set_index('Time', inplace=True)

In [268]:
from_date_filter = pd.to_datetime(from_date).tz_localize('Europe/Helsinki')

if quoter == 4:
    to_date_filter = f'{str(int(Y) + 1)}-01-01 01:00:00+02:00'
    to_date_filter = pd.to_datetime(to_date_filter).tz_convert('Europe/Helsinki')
else:
    to_date_filter = pd.to_datetime(to_date).tz_localize('Europe/Helsinki')
    to_date_filter = to_date_filter.replace(hour=23, minute=59, second=59)

print(from_date_filter,'\n',to_date_filter)

2022-10-01 00:00:00+03:00 
 2023-01-01 01:00:00+02:00


In [269]:
date_range = pd.date_range(start=from_date_filter, end=to_date_filter, freq='S')
missingSeconds = date_range[~date_range.isin(df.index)].value_counts().sum()

df = df.reindex(date_range)
print(f'Number of missing seconds: {missingSeconds}', end='\n\n\n')
df.reset_index(inplace=True, names="Time")
df['Value'].fillna(-1, inplace=True)
print(df[df['Value']==-1].head(5))

Number of missing seconds: 1812


                           Time  Value
7230  2022-10-01 02:00:30+03:00   -1.0
50431 2022-10-01 14:00:31+03:00   -1.0
50432 2022-10-01 14:00:32+03:00   -1.0
50433 2022-10-01 14:00:33+03:00   -1.0
50434 2022-10-01 14:00:34+03:00   -1.0


In [270]:
if missingSeconds == (df[df['Value'] == -1].value_counts().sum()):
    print('Test Ok')
else:
    print('Test Failed')

Test Ok


## Convert to Swedish time

In [271]:
df['Time'] = df['Time'].dt.tz_convert('Europe/Stockholm')
print(df.head(5))
print(df.tail(5))

                       Time     Value
0 2022-09-30 23:00:00+02:00  49.98771
1 2022-09-30 23:00:01+02:00  49.98478
2 2022-09-30 23:00:02+02:00  49.98796
3 2022-09-30 23:00:03+02:00  49.98621
4 2022-09-30 23:00:04+02:00  49.98732
                             Time     Value
7955996 2022-12-31 23:59:56+01:00  49.96575
7955997 2022-12-31 23:59:57+01:00  49.96507
7955998 2022-12-31 23:59:58+01:00  49.96166
7955999 2022-12-31 23:59:59+01:00  49.96383
7956000 2023-01-01 00:00:00+01:00  49.95928


In [272]:
from_date_filter = pd.to_datetime(from_date).tz_localize('Europe/Stockholm')
to_date_filter = pd.to_datetime(to_date).tz_localize('Europe/Stockholm')
to_date_filter = to_date_filter.replace(hour=23, minute=59, second=59)
if quoter == 1:
    df = df[(df['Time'] >= from_date_filter) & (df['Time'] <= to_date_filter)]
else:
    df = df[df['Time'] <= to_date_filter]
print(df.head(5))
print(df.tail(5))

                       Time     Value
0 2022-09-30 23:00:00+02:00  49.98771
1 2022-09-30 23:00:01+02:00  49.98478
2 2022-09-30 23:00:02+02:00  49.98796
3 2022-09-30 23:00:03+02:00  49.98621
4 2022-09-30 23:00:04+02:00  49.98732
                             Time     Value
7955995 2022-12-31 23:59:55+01:00  49.96766
7955996 2022-12-31 23:59:56+01:00  49.96575
7955997 2022-12-31 23:59:57+01:00  49.96507
7955998 2022-12-31 23:59:58+01:00  49.96166
7955999 2022-12-31 23:59:59+01:00  49.96383


### Test

In [273]:
if df.Value.count() == len(date_range) :
    print('Test passed')
else :
    print(df.Value.count() - len(date_range), '\nOne hour will always be removed if your data contains January (3600 seconds).')

-1 
One hour will always be removed if your data contains January (3600 seconds).


## Output

#### Save the data to a new csv

In [274]:
folder_name = 'processed_files'
file_name = output_filename

if not os.path.exists(folder_name):
    os.makedirs(folder_name)

file_path = os.path.join(folder_name, file_name)
df.to_csv(file_path, index=False)

#### Save to logfile

In [275]:
folder_name = 'log'
file_name = 'log.csv'
file_path = os.path.join(folder_name, file_name)

columns = ['Index', 'DateFrom', 'DateTo', 'NULL', 'NaN', 'Exact duplicates', 'Time duplicates', 'Zero Values', 'White Space', 'Added missing seconds']

new_data = {'Index': output_filename, 'DateFrom': from_date, 'DateTo': to_date, 'NULL': nrNull, 'NaN': nrNaN, 'Exact duplicates': num_duplicated, 'Time duplicates': num_duplicated_time, 'Zero Values': zeroValues, 'White Space': whiteSpaces, 'Added missing seconds': missingSeconds}

if not os.path.exists(file_path):
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
    log_df = pd.DataFrame(columns=columns)
else:
    log_df = pd.read_csv(file_path)

index_exists = (log_df['Index'] == new_data['Index']).any()

if index_exists:
    log_df.loc[log_df['Index'] == new_data['Index']] = [new_data[col] for col in columns]
else:
    new_row = pd.DataFrame([new_data], columns=columns)
    log_df = pd.concat([log_df, new_row], ignore_index=True)

log_df.to_csv(file_path, index=False)

In [276]:
display(log_df)

,Index,DateFrom,DateTo,NULL,NaN,Exact duplicates,Time duplicates,Zero Values,White Space,Added missing seconds
0,2020-Q1_seconds.csv,2020-01-01,2020-03-31,0,0,12,0,0,0,11939
1,2020-Q2_seconds.csv,2020-04-01,2020-06-30,0,0,9,6,0,0,17708
2,2020-Q3_seconds.csv,2020-07-01,2020-09-30,0,0,7,2,0,0,536994
3,2020-Q4_seconds.csv,2020-10-01,2020-12-31,0,0,13,0,0,0,119584
4,2021-Q1_seconds.csv,2021-01-01,2021-03-31,0,0,12,0,0,0,8304
5,2021-Q2_seconds.csv,2021-04-01,2021-06-30,0,0,11,2,0,0,9535
6,2021-Q3_seconds.csv,2021-07-01,2021-09-30,0,0,9,6,0,0,3218
7,2021-Q4_seconds.csv,2021-10-01,2021-12-31,0,0,13,0,0,0,1762
8,2022-Q1_seconds.csv,2022-01-01,2022-03-31,0,0,10,4,0,0,128146
9,2022-Q2_seconds.csv,2022-04-01,2022-06-30,0,0,5,14,0,0,2859
